'''
OBJECTIVES:
1. Build WRS system
2. Build Structural BMP Solution evaluator
3. Identify minimum BMP solution front for:
   individual facilities
   facilities w/in departments
   facilities w/in city
   
PYTHON VERSION: 3.6.3  
SQLALCHEMY VERSION: 1.1.13

'''

In [1]:
import winsound
import pandas as pd
'''
Define basic SQLAlchemy items:
    declarative base object
    connection object
    session object
    DB tables
'''
#SQLAlchemy library items:
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, String
from sqlalchemy import update, insert
from sqlalchemy import and_ #used in query.filter() to joing multiple where clauses
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship #http://docs.sqlalchemy.org/en/latest/orm/basic_relationships.html#relationship-patterns
from sqlalchemy import inspect

from SQLA_Base import Base #module containing declarative_base
from SQLA_conn_man import session, engine #module handling db and connection creation 

#Table definitions as SQLA classes:
from SQLA_DB_base_bmp_feasibility_test_results import Base_BMP_Feasibility_Test_Results as BBFTR
from SQLA_DB_base_bmp_feasibility_test_definitions import Base_BMP_Feasibility_Test_Definitions as BBFTD
from SQLA_DB_base_bmps import Base_BMPs
from SQLA_DB_combo_bmps import Combo_BMPs
from SQLA_DB_combo_bmp_feasibility_test_results import Combo_BMP_Feasibility_Test_Results as CBFTR
from SQLA_DB_expressions import Expressions
from SQLA_DB_facility_chars import Facility_Chars
from SQLA_DB_facility_monthly_rain import Facility_Monthly_Rain
from SQLA_DB_facility_risks import Facility_Risks
from SQLA_DB_facility_type_has_nel import Facility_Type_Has_NEL
from SQLA_DB_facility_types import Facility_Types
from SQLA_DB_feasibility_test_questions import Feasibility_Test_Questions as FTQ
from SQLA_DB_nel_sample_classes import NEL_Sample_Classes
from SQLA_DB_existing_pollutant_concentrations import Existing_Pollutant_Concentrations as ExPollConcs
from SQLA_DB_pollutant_removal_rates import Pollutant_Removal_Rates as PRR
from SQLA_DB_wrs_pollutant_risks import WRS_Pollutant_Risks
Base.metadata.create_all(engine, checkfirst=True) #create SQLA classes

'''
Dictionary of "SQLAlchemy where clause lambda functions" that importCSV uses to test record uniqueness.
used as the where clause in sqlalchemy queries, updates and deletes 
Form:
    {TableName:Lambda Function, TableName:Lambda Function, ...}
    
    TableName is the table name we want to define uniqueness test for
    Lambda Function is a SQLAlchemy query used to test record uniqueness. The function can take on any form 
        but must be made to evaluate the CSV row passed as a dictionary (CSVRowDict in this explanation):
        CSVRowDict: {FieldName:CSVColValue, DBTableFieldName:CSVColValue...} 
            Where: DBTableFieldName is the name of the field associated with the value at CSVColValue on the current row
                   CSVColValue: a value in the CSV's current row+column corresponding to the DBTableFieldName 
        *this assumes that field names are unique across table. if not, then method fails (maybe need to extend method?)
    FALSE: indicates that db table doesn't impose uniqueness on its records (other than its record id being unique)
        
e.g.: lambda myRowVal: Base.metadata.tables['people'].c['name'] == CSVRowDict['name']
        using lambda function in query will search for CSVRowDict's value for 'name' in the table people, field name 
if table has no record uniqueness requirement, then enter: TableName:False
'''
unqTests = {
    'facility_chars': lambda CSVRowDict: Base.metadata.tables['facility_chars'].c['Fac_Name'] == CSVRowDict['Fac_Name'],
    'facility_monthly_rain': False, #DB schema does not impose uniqueness on records in this table
    'facility_type_has_nel': False,
    'facility_risks': False,
    'facility_types': lambda CSVRowDict: Base.metadata.tables['facility_types'].c['Fac_Type'] == CSVRowDict['Fac_Type'],
    'nel_sample_classes': lambda CSVRowDict: Base.metadata.tables['nel_sample_classes'].c['nel_column']==CSVRowDict['nel_column'],
    'existing_pollutant_concentrations': False, #uniqueness not imposed for records in this table.
    'wrs_pollutant_risks': False #DB schema does not impose uniqueness on records in this table
}

import SQLA_main as SQLA_main #import main SQLAlchemy functions



Clearing old DB


In [2]:
'''
Define other custom modules

'''
import mod_Base_BMP_Eval as BBMP_Eval
import mod_Combo_BMP_Eval as CBMP_Eval
import mod_EffluentLimit as EffLim
import mod_expression as Expr
import mod_importSpecial as importSpecial #special import functions are defined here
import mod_importCSV as importCSV #generic CSV importer ****IMPORTANT NOTE: function assumes csv in the utf-8-sig file format. weird things happen if its not in this format!!!


In [3]:
#import feasibillity questions, build feasibility expressions
importSpecial.importFeasibilityQuestionsCSV('Input_Files\\feasibility_test_questions.csv') 

#import base bmp information including:
  #1. imports definitions for cip costs, o&m costs, and BMP sizing to the expressions table
  #2. imports pollutant removal rates into pollutant_removal_rates table
  #3. creates a record in the base_bmps table using (1) and (2)
  #4. feasibility tests
importSpecial.importBaseBMPsCSV('Input_Files\\bmp_lego_piece.csv') 

#IMPORT BASIC FACILITY CHARS:
    #!!!!IMPORTANT!!!! This import must occur before other facility specific data is imported!
print ('\nImporting facility characteristics:')
importCSV.importCSV('Input_Files\\facility_chars.csv', unqTests)

#IMPORT PBP Appendix A1 data
print ('\nImporting PBP Appendix A1 data:')
importCSV.importCSV('Input_Files\\pbp_appxa1.csv', unqTests)

#IMPORT FACILITY RAINFALL EXTRACTED FROM http://rainfall.geography.hawaii.edu/downloads.html
print ('\nImporting Facility Rainfall Data:')
importCSV.importCSV('Input_Files\\FacilityRainfallData.csv', unqTests)

#IMPORT EFFLUENT LIMITS EXISTANCE FOR FACILITY TYPES: (either by Priority Based Plan, Table 3 or as City operational assignment)
#IF CSV HEADRS SETUP CORRECTLY, THEN THIS INSERTS NEL EXISTANCE DATA (0 OR 1) TO WRS_POLLUTANT TABLE 
#AND USES THE FACILITY_TYPE_HAS_NEL TO ASSOCIATE RECORD WITH FACILITY TYPE
print ('\nImporting Facility Type Has Effluent Limits:') #import into wrs_pollutant_risks table
importCSV.importCSV('Input_Files\\nel_exists_facility_types.csv', unqTests)

#IMPORT NEL CLASSIFICATION DATA (from PBP Appendix L)
print ('\nImporting NEL Classes')
importCSV.importCSV('Input_Files\\nel_pbp_appxl.csv', unqTests)

#IMPORT FACILITY RISKS:
print ('\nImporting Facility Risks')
#for future implementation:
    #The current process inserts fac risk and update existing_fac_char_id in Facility_chars table. this process thus creates
#dead records. a more sophisticated approach using sophisticated lambda function in unqTests would fix this
importCSV.importCSV('Input_Files\\facility_risks.csv', unqTests)

# #IMPORT FACILITY SAMPLING DATA
 #!!!IMPORTANT!!!! For now, we make none detects = 0 BUT this must be changed to detection limit, per DOH guidance.
print ('\nImporting Facilty Sampling data:')
importCSV.importCSV('Input_Files\\sample_data.csv', unqTests)


# for now, since we're developing, delete out all except 1st 2 facilities.
# n = 5
# session.query(ExPollConcs).filter(ExPollConcs.facility_id >n).delete(synchronize_session = False) #http://docs.sqlalchemy.org/en/latest/orm/query.html#sqlalchemy.orm.query.Query.delete
# session.query(Facility_Chars).filter(Facility_Chars.id >n).delete(synchronize_session = False) #http://docs.sqlalchemy.org/en/latest/orm/query.html#sqlalchemy.orm.query.Query.delete
# session.commit #we chose not to sync session so need to commit before proceeding to requery or else you may get unpredictable resutls

session.commit()
winsound.Beep(250,1000)

Reading csv for import to Feasibility Questions

Reading csv record: Feas-1

Reading csv record: Feas-2

Reading csv record: Feas-3

Reading csv record: Feas-4

Reading csv record: Feas-5

Reading csv record: Feas-6

Reading csv record: Feas-7

Reading csv record: Feas-8

Reading csv record: Feas-9

Reading csv record: Feas-10

Reading csv record: Feas-11

Reading csv record: Feas-12

Reading csv record: Feas-13

Reading csv record: Feas-14

Reading csv record: Feas-15

Reading csv record: Feas-16

Reading csv record: Feas-17

Reading csv record: Feas-18

Reading csv record: Feas-19

Reading csv record: Feas-20

Reading csv record: Feas-21
Reading csv for import to base bmp tables

Reading csv record: Hydrodynamic Separation
Reading pollutant removal rate info...
Linking feasibility tests w/ base bmp: 1
Removed:  0  old feasibility test defs for the base bmp
Added feasibility test def as record:  1
Added feasibility test def as record:  2
Added feasibility test def as record:  3
Added 

In [4]:
# # EVALUATE BASE BMP FEASIBILITY at each facility
# # Write results to the base_bmp_feasibility_test_results table.

# print('\n******Evaluating Base BMP feasibility at facilities.******')
# ShowCalculations = False #flag indicating if steps should be outputted
# Expr.ResetEvalErrorCount() #RESET EXPRESION EVALUATOR ERROR COUNT
# for aFac in session.query(Facility_Chars):
#     if ShowCalculations: print ('\n***Evaluating base bmp feasibiilty tests for facility: ', aFac.Fac_Name), ' ***'
#     myBMPs = session.query(Base_BMPs)
#     for aBMP in myBMPs:
#         if ShowCalculations:print ('\nEvaluating feasibility of base_bmp: ', aBMP.bmp_name, ' ID: ', aBMP.id)
#         BBMP_Eval.Eval_base_bmp_feasibility_tests(aFac, aBMP, False)
# session.commit
# winsound.Beep(250,1000)
# print ('*****************************************************************')
# print ('* Completed evaluating Base BMP feasibility                     *')
# if Expr.CountEvalErrors() >0:
#     print (Expr.CountEvalErrors(), ' errors were encountered. Review output to identify location(s)')
#     print ('Hint: expression evaluation error lines are prefixed by: FAULT!!!! Error occured while evaluating expression:')
# else:
#     print ('No errors detected.')
# print ('*****************************************************************')

In [5]:
'''
Enter estimated pollutant concentrations into database's existing pollutant concentration table for facilities without 
actual sampling data. Use 1 of 2 methods:

Method 1: Use maximum concentration value sampled for period 2013-2017
          This method is for Permit Table 1 facilities only
          Method assumes we have already entered sampling data for into the database's existing pollutant concentration table

Method 2: Use data from an EMC study.
          This method is for facilities that are not on Permit Table 1
          
          
Defined Global Vars:
    pd_infieldExtrema: dataframe holding maximum concentrations that were measured. *exception is phMin. we use lowest observed ph
    
    
'''
pollLS = ['tss', 'turbidity', 'p', 'n', 'nn', 'an', 'og', 'cu', 'zn', 'fe', 'phmin', 'phmax'] #list of pollutant constituants we're trying to address


#get all existing pollutant concentrations obtained using the 'infield' sampling method
q = session.query(ExPollConcs).filter(ExPollConcs.sample_method == 'infield')
pd_Concs = pd.read_sql(q.statement,session.bind) 

#build pd_infieldExtreama by making a dictionary of maximum sample results for each constiuent
dict_extrema = {'c_' + Constituent: pd_Concs.loc[:,'c_' + Constituent].max() for Constituent in pollLS}
dict_extrema['c_phmin'] = pd_Concs.loc[:,'c_phmin'].min() #phMin is exception to above. we want min. phMin value
#use dictionary to build pd_infieldExtrema dataframe
pd_infieldExtrema = pd.DataFrame([dict_extrema])
display(pd_infieldExtrema)

#Apply Method1 to all unsampled Table 1 facilities
#we need to get all Table 1 facilities that are not currently in the existing pollutant concentration table
#############################################################################################################
'''
To be sure we're starting fresh, let's remove any records in ExPollConcs that:
1. Were not obtained directly from field samples (i.e. sample_method != 'infield)
2. Were obtained from field samples, but are not Table 1 facilities (i.e. we shouldn't be looking at their  sample results)
'''
#delete all pollutant concentration table records that are not from infield sampling.
session.query(ExPollConcs).filter(ExPollConcs.sample_method != 'infield').delete(synchronize_session = False)
#delete all pollutant concentration table records that are not for Table 1 facilities
#for some reason bulk delete's not working. so let's use a loop to work around it.
for rec in session.query(ExPollConcs.id).filter(ExPollConcs.facility_id == Facility_Chars.id).filter(Facility_Chars.Permit_Table != 'Table 1'):
    session.query(ExPollConcs).filter(ExPollConcs.id == rec[0]).delete(synchronize_session = False)


#now build query that identifies all Table 1 facilities that are not in ExPollConcs
subq = session.query(ExPollConcs.facility_id.distinct()).order_by(ExPollConcs.facility_id).all()
ls_sq = [i[0] for i in subq if i[0] is not None] #list comprehension to produce list of all facility_id in ExPollConcs table
#get list of Table 1 facilities not in ExPollConcs:
tpl_q = session.query(Facility_Chars.id).filter(Facility_Chars.Permit_Table == 'Table 1').filter(Facility_Chars.id.notin_(ls_sq)).all()
ls_FacIDs = [i[0] for i in tpl_q] #write query tuple to list    
#make a list of Table 1 facs not in ExPollConcs (a list of dicts). also include extrema conc. values.  
ls_dict_pd = [{**{'facility_id': FacID, 'sample_method': 'sim_MaxType', 'sample_date':'12/31/2016'}, **dict_extrema} for FacID in ls_FacIDs]
#write list to database:
ExPollConcs_meta = Base.metadata.tables['existing_pollutant_concentrations']
ExPollConcs_id_meta = ExPollConcs_meta.c['id']
for dict_temp in ls_dict_pd:
    SQLA_main.insertRec(ExPollConcs_meta,dict_temp)
session.commit()
#for future implementation: write dict -> dataframe -> db(using sqla):
    # pd_temp.to_sql('existing_pollutant_concentrations', engine, if_exists='append', index = False)
    #http://docs.sqlalchemy.org/en/latest/faq/performance.html#i-m-inserting-400-000-rows-with-the-orm-and-it-s-really-slow
    #https://stackoverflow.com/questions/31997859/bulk-insert-a-pandas-dataframe-using-sqlalchemy
q = session.query(ExPollConcs)
pd.read_sql(q.statement,session.bind) 

,c_an,c_cu,c_fe,c_n,c_nn,c_og,c_p,c_phmax,c_phmin,c_tss,c_turbidity,c_zn
0,10.5,105.0,19500.0,27.25,3.68,59.0,10.3,8.7,6.01,2010.0,4181.0,730.0


,id,facility_id,sample_method,sample_point_name,sample_date,c_tss,c_turbidity,c_p,c_n,c_nn,c_an,c_og,c_cu,c_zn,c_fe,c_phmin,c_phmax
0,4,15,infield,CAESHAL,4/20/2017,NaN,22.00,0.300,2.130,0.300,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5,15,infield,CAESHAL,2/11/2017,NaN,21.00,0.146,5.070,0.210,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6,15,infield,CAESHAL,12/8/2016,30.0,28.10,0.200,1.240,0.370,0.000,0.0,NaN,NaN,NaN,7.90,7.90
3,7,15,infield,CAESHAL,6/17/2016,23.0,24.30,0.182,0.850,0.000,0.000,0.0,NaN,NaN,NaN,6.87,6.87
4,8,15,infield,CAESHAL,5/2/2014,42.5,29.30,0.230,1.161,0.101,0.050,4.9,NaN,NaN,NaN,7.65,7.65
5,9,15,infield,CAESHAL,3/9/2013,53.5,NaN,0.337,2.485,0.545,0.160,NaN,NaN,NaN,NaN,NaN,NaN
6,10,15,infield,CAESHAL,1/27/2013,NaN,23.60,NaN,NaN,NaN,NaN,4.7,NaN,NaN,NaN,7.76,7.76
7,11,21,infield,CWCSMHAL,5/2/2014,30.5,17.40,0.161,1.152,0.092,0.050,4.8,NaN,NaN,NaN,7.55,7.55
8,12,21,infield,CWCSMHAL,1/27/2013,233.0,240.00,0.558,1.243,0.203,0.068,4.7,NaN,NaN,NaN,7.45,7.45
9,13,8,infield,CDRMHAL,4/20/2017,NaN,29.00,0.500,1.690,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
#Estimate Pollutant Effluent Limits
'''
Estimate the Numeric Effluent Limits (NELs) for each facility.
Return wet and dry season NELs in 2 separate dataframes:
    pd_FacsNELs_Wet & pd_FacsNELs_Dry
Estimate NELs using the EffLim module's GetNELs function call.
 The GetNELs function call will differentiate between wet and dry season limits
 (if limits are the same between wet & dry season, then the same limit will be placed into the wet and dry
  dataframes.)
 The GetNEls function calculates a pollutant constituent NEL using this formula:
    NEL = fTypeHas_NEL * SampleClass_NEL
    Where:
      fTypeHas_NEL is a [0,1] value from PBP Table 3, based on facility type (stored in SQLA_DB_facility_type_has_nel)
      SampleClass_NEL is pollutant concentration based on facility's sample class, based on PBP Appendix L
'''
pd_FacsNELs_Wet, pd_FacsNELs_Dry = pd.DataFrame(),  pd.DataFrame() #initialize wet and dry season nel dataframes 
for recFac in session.query(Facility_Chars): #do the following for each facility:
    wet,dry = EffLim.GetNELs(recFac,False) #Get Wed & Dry NELs by calculating: NEL = fTypeHas_NEL * SampleClass_NEL
#     if wet is not None:
    pd_FacsNELs_Wet = pd.concat([pd_FacsNELs_Wet, wet]) #write wet NELs to pd_FacsNELs_Wet
#     if dry is not None:
    pd_FacsNELs_Dry = pd.concat([pd_FacsNELs_Dry, dry]) #write dry NELs to pd_FacsNELs_Dry

print('Wet NELs:')
display(pd_FacsNELs_Wet)
print('Dry NELs:')
display(pd_FacsNELs_Dry)


Wet NELs:


,wrs_tss,wrs_turbidity,wrs_p,wrs_n,wrs_nn,wrs_an,wrs_og,wrs_cu,wrs_zn,wrs_fe,wrs_phmin,wrs_phmax
Facility_ID,,,,,,,,,,,,
1,50.0,15.00,0.10,0.52,0.180,NaN,15.0,NaN,NaN,NaN,5.5,8.0
2,50.0,15.00,0.10,0.52,0.180,NaN,15.0,NaN,NaN,NaN,5.5,8.0
3,50.0,15.00,0.10,0.52,0.180,NaN,15.0,6.0,22.0,NaN,5.5,8.0
4,NaN,3.00,0.05,0.35,0.020,0.0130,15.0,2.9,95.0,NaN,7.0,8.6
5,50.0,15.00,0.10,0.52,0.180,NaN,15.0,NaN,NaN,NaN,5.5,8.0
6,NaN,NaN,NaN,NaN,NaN,NaN,15.0,6.0,22.0,NaN,5.5,8.0
7,NaN,0.50,0.03,0.18,0.010,0.0050,15.0,2.9,95.0,NaN,7.6,8.6
8,50.0,15.00,0.10,0.52,0.180,NaN,15.0,NaN,NaN,NaN,5.5,8.0
9,NaN,1.25,0.04,0.25,0.014,0.0085,15.0,NaN,NaN,NaN,7.6,8.6


Dry NELs:


,wrs_tss,wrs_turbidity,wrs_p,wrs_n,wrs_nn,wrs_an,wrs_og,wrs_cu,wrs_zn,wrs_fe,wrs_phmin,wrs_phmax
Facility_ID,,,,,,,,,,,,
1,30.0,5.50,0.06,0.38,0.090,NaN,15.0,NaN,NaN,NaN,5.5,8.0
2,30.0,5.50,0.06,0.38,0.090,NaN,15.0,NaN,NaN,NaN,5.5,8.0
3,30.0,5.50,0.06,0.38,0.090,NaN,15.0,6.0,22.0,NaN,5.5,8.0
4,NaN,3.00,0.05,0.35,0.020,0.0130,15.0,2.9,95.0,NaN,7.0,8.6
5,30.0,5.50,0.06,0.38,0.090,NaN,15.0,NaN,NaN,NaN,5.5,8.0
6,NaN,NaN,NaN,NaN,NaN,NaN,15.0,6.0,22.0,NaN,5.5,8.0
7,NaN,0.50,0.03,0.18,0.010,0.0050,15.0,2.9,95.0,NaN,7.6,8.6
8,30.0,5.50,0.06,0.38,0.090,NaN,15.0,NaN,NaN,NaN,5.5,8.0
9,NaN,1.25,0.04,0.25,0.014,0.0085,15.0,NaN,NaN,NaN,7.6,8.6


In [7]:
import pandas as pd
import numpy as np
import math
import datetime

In [8]:
#let's work on calculating exceedance values

import mod_EffluentLimit as EffLim

#get all sampling data
q = session.query(ExPollConcs.id, ExPollConcs.facility_id.label('Facility_ID'), ExPollConcs.sample_date, 
        ExPollConcs.c_tss,
        ExPollConcs.c_turbidity,
        ExPollConcs.c_p,
        ExPollConcs.c_n,
        ExPollConcs.c_nn,
        ExPollConcs.c_an,
        ExPollConcs.c_og,
        ExPollConcs.c_cu,
        ExPollConcs.c_zn,
        ExPollConcs.c_fe,
        ExPollConcs.c_phmin,
        ExPollConcs.c_phmax  
         ).order_by(ExPollConcs.facility_id) #.filter(ExPollConcs.facility_id == FacID)
pd_Concs = pd.read_sql(q.statement,session.bind) 
#tidy up the sampling data
from datetime import datetime
pd_Concs['sample_date'] = pd.to_datetime(pd_Concs['sample_date'], format="%m/%d/%Y")
pd_Concs = pd_Concs.applymap(lambda x: float('nan') if x is None else x) #assign NaN values to any None element

#for each pollutant constituent, do the Exceedance Calculation = max(0,(Constituent Concentration - NEL))
# if no exceedance, then report 0. report NaN sample result is NaN
pd_FacExceedances = pd.DataFrame() #make an empty dataframe.  we will append to it.
for Facs in session.query(Facility_Chars.id).order_by(Facility_Chars.id):
    FacID = Facs[0]
    if (FacID in pd_Concs.Facility_ID.values) and (FacID in pd_FacsNELs_Dry.index) and (FacID in pd_FacsNELs_Wet.index):
        pd_temp = pd_Concs.loc[pd_Concs['Facility_ID'] == FacID] #slice facility id rows into a temp dataframe
        pd_temp.is_copy = False #acknowledge that pd_temp is NOT a copy of pd_Concs. but intended to be treated as new dataframe
        for Constituent in pollLS:
            pd_temp['c_' + Constituent]  = pd_Concs.apply(lambda row: #for each row in pd_Concs:
                                     EffLim.ExceedanceCalc(row, Constituent, FacID, pd_FacsNELs_Wet, pd_FacsNELs_Dry, True), axis = 1)
#         display(pd_temp)
        pd_FacExceedances = pd.concat([pd_FacExceedances,pd_temp])
print('Concentrations in excess of wet/dry season NELs')
display(pd_FacExceedances)

Concentrations in excess of wet/dry season NELs


,id,Facility_ID,sample_date,c_tss,c_turbidity,c_p,c_n,c_nn,c_an,c_og,c_cu,c_zn,c_fe,c_phmin,c_phmax
0,35,1,2017-04-19,72.0,NaN,NaN,1.910,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00
1,36,1,2017-02-11,9.0,0.0,0.000,0.060,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.30
2,37,1,2016-12-04,30.0,56.2,0.200,0.390,0.000,NaN,0.0,NaN,NaN,NaN,0.00,0.20
3,38,1,2016-06-17,53.0,75.9,0.190,0.560,0.110,NaN,0.0,NaN,NaN,NaN,0.00,0.00
4,39,1,2015-02-20,0.0,2.5,0.076,1.310,0.010,NaN,0.0,NaN,NaN,NaN,0.00,0.54
5,40,1,2014-04-13,0.0,0.0,0.144,1.517,0.067,NaN,0.0,NaN,NaN,NaN,0.00,0.00
6,41,1,2013-03-09,113.0,9.4,0.055,0.679,0.059,NaN,0.0,NaN,NaN,NaN,0.00,0.09
7,80,2,2017-04-20,NaN,0.0,NaN,0.260,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,81,2,2017-01-21,0.0,16.0,0.000,1.060,0.000,NaN,0.0,NaN,NaN,NaN,0.00,0.00
9,82,2,2016-05-05,0.0,0.0,0.006,2.292,0.122,NaN,0.0,NaN,NaN,NaN,0.00,0.00


In [12]:
'''
CALCULATE AGE FACTOR WEIGHTED AVERAGE FOR EACH CONSTITUENT:

Age factor acknowledges fact that more recent samples are a better representation of facility pollutant discharge 
(i.e. sampling data) and housekeeping-operations (i.e. inspections) realities. But, historic data as a whole also tells part 
of story (i.e. we want to dampen whipsaw effects that may occur if we only considered most recent data).

AF = exp(-SampleRank)
SampleRank = Newest sample = 1
              Second Newest sample = 2
              ...
              nth Newest Sample = n (out of n samples)
'''
#calculate age factor weighted averages for each constituent of each facility.
#write these averages into a dataframe called pd_AFWFacExceedances
#use exceedance values in pd_FacExceedances
#we will process each facility separately. The results will be added to the pd_AFWFacExceedances dataframe
pd_AFWFacExceedances = pd.DataFrame() #make an empty dataframe.  we will append to it.
ShowCalculations = False
UnqFacIDs = pd_FacExceedances.Facility_ID.unique() #get unique facility IDs
for FacID in UnqFacIDs:
    pd_temp = pd_FacExceedances.loc[pd_FacExceedances['Facility_ID'] == FacID].set_index('sample_date') #write current fac's data to temp df
    pd_AFWFacExceedances = pd.concat([pd_AFWFacExceedances,EffLim.Calc_AFWtdExceedances(pd_temp,pollLS,ShowCalculations)]) #calculate age factor weighted averages
print ('Age Factor Weighted Averages:')
display(pd_AFWFacExceedances)

Age Factor Weighted Averages:


,Facility_ID,AFWtd_c_tss,AFWtd_c_turbidity,AFWtd_c_p,AFWtd_c_n,AFWtd_c_nn,AFWtd_c_an,AFWtd_c_og,AFWtd_c_cu,AFWtd_c_zn,AFWtd_c_fe,AFWtd_c_phmin,AFWtd_c_phmax
0,1,52.064543,19.729711,0.067223,1.296165,0.029425,NaN,0.000000,NaN,NaN,NaN,0.000000,0.093351
0,2,0.961758,3.745947,0.004876,0.637805,0.031165,NaN,0.000000,NaN,NaN,NaN,0.000000,0.000000
0,3,696.752557,515.665239,1.231643,1.193739,0.687195,NaN,0.000000,44.093122,542.332084,NaN,0.000000,0.100000
0,4,NaN,49.764373,1.282323,1.405886,0.499160,0.074351,0.000000,63.327487,224.735333,NaN,0.009474,0.000000
0,5,180.960566,199.986459,1.752511,17.437221,0.004532,NaN,1.421297,NaN,NaN,NaN,0.000000,0.000000
0,6,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,81.689667,112.670680,NaN,0.000000,0.086681
0,7,NaN,15.400000,0.860000,3.290000,2.820000,0.103000,0.000000,16.000000,0.000000,NaN,0.000000,0.000000
0,8,14.732924,16.599986,0.691190,0.829885,0.006187,NaN,0.000000,NaN,NaN,NaN,0.000000,0.070236
0,9,NaN,4179.750000,10.260000,27.000000,3.666000,10.491500,44.000000,NaN,NaN,NaN,1.590000,0.100000
0,10,NaN,4178.000000,10.250000,26.900000,3.660000,10.487000,44.000000,NaN,NaN,NaN,0.990000,0.100000


In [71]:
#calculate monthly runoff volumes in cubic feet for each facility.
#hold data in dataframe called pd_RunoffVols
import calendar

#get facility imperviousness and area. order by Facility_ID so it's given in same order as monthly rain data dataframe
q_facDat = session.query(Facility_Chars.id.label('Facility_ID'), 
                         Facility_Chars.Drainage_Area_Acres.label('Drn_Area'), 
                         Facility_Chars.Imperv.label('Imperv')).order_by('Facility_ID')
pd_facDat = pd.read_sql(q_facDat.statement,session.bind)

#get monthly rain data for each facility. order by facility_id so order matches facility data dataframe
q_rain = session.query(Facility_Chars.id.label('Facility_ID'), Facility_Monthly_Rain).filter(
    Facility_Chars.facility_monthly_rain_id == Facility_Monthly_Rain.id).order_by('Facility_ID')
pd_rainDat = pd.read_sql(q_rain.statement,session.bind)

#create a new dataframe to hold rain volumes
pd_RunoffVols = pd_facDat.loc[:,['Facility_ID']] #put facilities into the new dataframe
#now calculate volumes for each month:
for mo in range(1,13):
    pd_RunoffVols[calendar.month_name[mo]] = pd.DataFrame(pd_facDat['Drn_Area'] * pd_facDat['Imperv'] * pd_rainDat[calendar.month_name[mo]])
display(pd_RunoffVols)
    



,Facility_ID,January,February,March,April,May,June,July,August,September,October,November,December
0,1,325440.00000,369600.000000,388800.000000,275520.000000,173760.00000,154560.0000,158400.000000,152640.000000,188160.000000,286080.000000,368640.000000,432000.000000
1,2,87373.57253,63429.853135,62169.657377,26254.078284,33815.25283,24783.8499,24993.882527,20163.132122,27094.208789,67840.538287,90734.094551,87163.539904
2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
#CREATE COMBO BMPS USING BASE BMPS
#ALL POSSIBLE COMBOS WILL BE CREATED AND ADDED TO THE COMBO_BMPS TABLE
#MAXIMUM POLLUTANT REMOVAL RATES ARE DETERMINED BY IDENTIFYING 
#  THE BASE_BMP IN THE COMBO THAT PROVIDES THE HIGHEST REMOVAL RATE FOR A GIVEN POLLUTANT

import time
print ('get a coffee...this one takes a while!')
start_time = time.time()
CBMP_Eval.Make_ALL_bmp_base_option_combos()
session.commit()
print ('--- %s execution time in seconds ---' % (time.time() - start_time))

get a coffee...this one takes a while!
 Making BMP Combos of length: 1
 Find max pollutant removal rates for each BMP Combo of length:  1
  Made  13  combos
 Making BMP Combos of length: 2
 Find max pollutant removal rates for each BMP Combo of length:  2
  Made  78  combos
 Making BMP Combos of length: 3
 Find max pollutant removal rates for each BMP Combo of length:  3
  Made  286  combos
 Making BMP Combos of length: 4
 Find max pollutant removal rates for each BMP Combo of length:  4
  Made  715  combos
 Making BMP Combos of length: 5
 Find max pollutant removal rates for each BMP Combo of length:  5
  Made  1287  combos
 Making BMP Combos of length: 6
 Find max pollutant removal rates for each BMP Combo of length:  6
  Made  1716  combos
 Making BMP Combos of length: 7
 Find max pollutant removal rates for each BMP Combo of length:  7
  Made  1716  combos
 Making BMP Combos of length: 8
 Find max pollutant removal rates for each BMP Combo of length:  8
  Made  1287  combos
 Making

In [11]:
'''
Identify the feasible bmp combinations for each facility
Use base bmp feasibility results for each facility.
Put results into the combo_bmp_feasibility_test_results table
'''
import itertools     #https://docs.python.org/3/library/itertools.html    
import pandas as pd

from sqlalchemy import and_

def _Make_bmp_fingerprint(base_BMP_components):
    #create fingerprint of the passed list of base_bmp_ids
    #fingerprint is just a | separated list of ids of the base bmps that make up the combo bmp
    #corresponds to bmp_options table's bmp_fingerprint field
    #FORMAT: |bmp_option_base_component_id||bmp_option_base_component_id| w/ id's given in ascending order
    fingerprint = '|' + '|'.join(str(id) + '|' for id in base_BMP_components)
    return fingerprint

def Eval_FacBMPCombo(pd_basebmps, myFacility, bmpCombo):
    '''
    input:
        pdbasebmps: pandas built from a BBMP_Eval.evalFacility_BaseBMP dictionary list
                    assme that pandas is passed in w/ index is set as base_bmp_id
        myFacility: SQLA fac_chars record
        bmpCombo: list of base_bmp_ids that make up this combo
    
    #retrieve previously computed combo removal rate
    #calculate combo cip and om cost, insert/update database
    #calculate wrs reduction, insert/update database

    #return as pandas    
    '''    
    #get combo bmp pollutant removal rates into pandas 
    q = session.query(Combo_BMPs.bmp_fingerprint, Combo_BMPs.id.label('combos_bmp_id'), PRR.id.label('PRR_id'),
          PRR.r_tss, PRR.r_turbidity, PRR.r_p, PRR.r_n, PRR.r_nn, PRR.r_an,
          PRR.r_og, PRR.r_cu, PRR.r_zn, PRR.r_fe, PRR.r_phmin, PRR.r_phmax
        ).filter(Combo_BMPs.bmp_fingerprint == _Make_bmp_fingerprint(bmpCombo)).filter(
        Combo_BMPs.bmp_option_removal_rate_id == PRR.id)  
    pd_rr = pd.read_sql(q.statement,session.bind) 

    #use information in pd_rr to get CBFTR_record - make new record if necessary
    myCBFTR = Base.metadata.tables['combo_bmp_feasibility_test_results']
    myCBFTR_id = SQLA_main.insertupdateRec(myCBFTR,{'facility_id':myFacility.id, 'combo_bmps_id':pd_rr['combos_bmp_id'][0]},
                and_(
        myCBFTR.c['facility_id'] == myFacility.id,
        myCBFTR.c['combo_bmps_id'] == pd_rr['combos_bmp_id'][0]
                    ))
    session.flush()
    
    print (myCBFTR_id)
    
    #calculate WRS reduction
#     myFac_exWRSData = session.query(
    
    

    #get costs in pandas
    sumCIP = sum(pd_basebmps.loc[bmp_id,'calc_cip_cost'] for bmp_id in bmpCombo)
    sumOM = sum(pd_basebmps.loc[bmp_id,'calc_om_cost'] for bmp_id in bmpCombo)
    pd_sums = pd.DataFrame([{'calc_cip_cost':sumCIP, 'calc_om_cost': sumOM}])

    #merge combo bmp's removal rates and costs into 1 dataframe
    return pd.concat([pd_rr, pd_sums], axis = 1)
    
    

def Eval_FacBMPOptions(myFacility):
    #a wrapper around Eval_FacBMPCombo
    print('\n***Evaluating feasible bmp combos for facility: ', aFac.Fac_Name, '***')
    print ('****Evaluating feasibile base bmps****')
    df = pd.DataFrame(BBMP_Eval.evalFacility_BaseBMP(aFac, False)).set_index('base_bmp_id')
    display (df)   
    print ('****These are the feasible base bmps. I\'ll use them to make combos:****')
    df = df.loc[df['is_feasible'] == 1]
    display (df)
    feas_ls = df.index #send feasible base bmp ids to list
#     print (feas_ls)
# from SQLA_DB_combo_bmps import Combo_BMPs
# from SQLA_DB_combo_bmp_feasibility_test_results import Combo_BMP_Feasibility_Test_Results as CBFTR
    for CBOLen in range (1, len(feas_ls)+1): #+1 so it's inclusive of last count
        for combo in  itertools.combinations(feas_ls,CBOLen):
            print ('Here is a summary of the combo: ', list(combo))
            display(Eval_FacBMPCombo(df,myFacility, list(combo)))

            
def Eval_All_FacBMPOptions():
    print ('Evaluating feasibile BMP Options for each facility:')
    for aFac in session.query(Facility_Chars):
        Eval_FacBMPOptions(aFac)
    
Eval_All_FacBMPOptions()
session.commit()

Evaluating feasibile BMP Options for each facility:


NameError: name 'aFac' is not defined

In [ ]:
# session.close()
# engine.dispose()

In [ ]:
# http://pythonhow.com/accessing-dataframe-columns-rows-and-cells/
import pandas as pd #import in pandas library
print ('#get csv data and read into pandas')
df1=pd.read_csv("http://pythonhow.com/wp-content/uploads/2016/01/Income_data.csv")
print (df1)
print ('#write new dataframe w/ index set to the "State" column in the csv')
df2=df1.set_index("State").copy()
print (df2)
print ('#extract a portion of the dataframe: States = Alaska to Arkansas; and Dates 2005:2007')
print (df2.loc["Alaska":"Arkansas","2005":"2007"])

print ('Get only certain States, using a list of states:')
getStates = ['Alaska', 'Arizona']
print (df2.loc[getStates])

print ('#slice a column:')
df2.loc[: , "2005"]
print ('get a cell:')
df2.loc['Alaska','2005']
print ('#get max of 2005 data')
print (df2.loc[:,'2005'].max())
print ('take 2005 column and put into list')
LS = df2['2005'].tolist() #this is a series. we use the .tolist() to convert from series to list
print (type(LS))


In [ ]:
df = pd.DataFrame({'col1' : [1.0] * 5, 
                   'col2' : [2.0] * 5, 
                   'col3' : [3.0] * 5 }, index = range(1,6),)
display(df)
df2 = pd.DataFrame({'col1' : [10.0] * 5, 
                    'col2' : [100.0] * 5, 
                    'col3' : [1000.0] * 5 }, index = range(1,6),)
display(df2)
df.mul(df2, 0) # element by element multiplication no problems

In [ ]:
import datetime

# xmin = datetime.datetime.strptime('1/1/2018', "%m/%d/%Y").date()
# xmax = datetime.datetime.strptime('5/6/2018', "%m/%d/%Y").date()

# xmin <= datetime.date(2018,1,5) <= xmax

#     Wet Season is from: January 1 through April 30 and November 1 through December 31
#     Dry Season is from: May 1 through October 31

SampleDate = datetime.date(2018,11,1)

#Wet Season 1:
if datetime.date(SampleDate.year, 1,1) <= SampleDate <= datetime.date(SampleDate.year, 4,30):
    print ('ws 1')
elif datetime.date(SampleDate.year, 5,1) <= SampleDate <= datetime.date(SampleDate.year, 10,31):
    print ('dry')
else:
    print ('ws 2')
    
    
import numpy as np    
# np.max([float('nan'),0])
np.max([0,float('nan')])

if math.isnan(10)

In [ ]:
#import the pandas library
import pandas as pd

ipl_data = {'Team': ['Riders', 'Riders', 'Devils', 'Devils', 'Kings',
         'kings', 'Kings', 'Kings', 'Riders', 'Royals', 'Royals', 'Riders'],
         'Rank': [1, 2, 2, 3, 3,4 ,1 ,1,2 , 4,1,2],
         'Year': [2014,2015,2014,2015,2014,2015,2016,2017,2016,2014,2015,2017],
         'Points':[876,789,863,673,741,812,756,788,694,701,804,690]}
df = pd.DataFrame(ipl_data)

display (df)

display (df.groupby('Team').groups)


import numpy as np


grouped = df.groupby('Year')
print (grouped['Points'].agg(np.max))

In [ ]:
df = pd.DataFrame({'col1': [1, 2], 'col2': [0.1, 0.2]},
                      index=['a', 'b'])
for idx in df.index:
    print (idx)
    print (df.loc[idx]['col2'])

In [ ]:
t1 = True
t2 = True
print (t1 and t2)